In [24]:
from simple_io import IO
import jieba
from jieba import posseg
from collections import Counter
import re,time
import json
import requests

appname = '网易彩票'
subtitle = '网易的彩票啊啊啊'
KeyWords = '彩票足球开奖六合彩体育直播篮球,彩六合双色球快比分助手北京福彩预测,全民竞猜三福利App体彩天天中,七星赛车足彩3福利彩票重庆中国球探大师3d吧,博彩足球彩票pk10新浪选五'


In [25]:
t1 = time.time()

In [26]:
n = 1
if len(appname) > 23 :
    print(str(n) + '、您的应用名称填写了 ' + str(len(appname)) + ' 个字符。\n 应用名称保持在23个字符左右或更少，让其在每个设备上都有最优的表现。')
    n += 1
if len(appname) < 7 :
    print(str(n) + '、您的应用名称填写了 ' + str(len(appname)) + ' 个字符。\n 您的应用名称过短，建议补充以增加关键词覆盖。例如：“腾讯视频-电视剧电影综艺体育直播视频播放器”')
    n += 1 


KeyWord = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "",KeyWords+subtitle+appname) 

def posseg_cut(subtitle, keyword):
    seg_list = posseg.cut(subtitle)
    for word,flag in seg_list:  
        if flag == 'n':   
            keyword += word
            keyword += ','
    return keyword[:-1]

def jieba_cut(subtitle, keyword):
    seg_list = jieba.cut_for_search(subtitle)
    for word in seg_list:
        keyword += word
        keyword += ','
    return keyword[:-1]

keyword = jieba_cut(KeyWord, '')
keyword = keyword.split(',')

result = str(n) + '、您的应用名称和关键词中，'
times = Counter(keyword)
for word in times.most_common():
    if word[1] > 1:
        result = result + "“" + word[0] + "”" + "出现了 " + str(word[1]) + " 次，"
result = result[:-1] + "。\n更换其他词，以覆盖更多的关键词。"
print(result)
n += 1

if len(KeyWords) < 100:
    print(str(n) + '、App Store 关键词可以填写 100 个字符，您的关键词仅填写了 '+ str(len(KeyWords))+' 字。\n补充至100个字符，以充分利用字符空间，增加可能覆盖的关键词数量。')

1、您的应用名称填写了 4 个字符。
 您的应用名称过短，建议补充以增加关键词覆盖。例如：“腾讯视频-电视剧电影综艺体育直播视频播放器”
2、您的应用名称和关键词中，“彩票”出现了 5 次，“网易”出现了 2 次，“六合”出现了 2 次，“足球”出现了 2 次，“福利”出现了 2 次。
更换其他词，以覆盖更多的关键词。
3、App Store 关键词可以填写 100 个字符，您的关键词仅填写了 87 字。
补充至100个字符，以充分利用字符空间，增加可能覆盖的关键词数量。


In [27]:
timeout = 4

def rank(word):
    url ='http://rest.appbk.com/word/get_word_hot_rank?n='+ word +'&cc=cn'
    try:
        resp_ = requests.get(url,timeout=timeout,verify = False)
        resp_2 = resp_.content
        resp_3 = resp_2.decode('utf-8')

        resp_list = json.loads(resp_3)
        return int(resp_list['rank'])
    except Exception as e:
        print(e)    
        return -1

In [28]:
def resnum(word):
    url ='http://rest.appbk.com/app/get_all_app_search_results?n='+ word +'&start=0&limit=30&cc=cn'
    try:
        resp_ = requests.get(url,timeout=timeout,verify = False)
        resp_2 = resp_.content
        resp_3 = resp_2.decode('utf-8')

        resp_list = json.loads(resp_3)
        return int(resp_list['num'])
    except Exception as e:
        print(e)    
        return -1

In [29]:
result = []
result2 = []
keyword = list(set(keyword))
for word in keyword:
    wordrank = rank(word)
    if wordrank > 4604:
        wordres = resnum(word)
        if wordres != -1:
            result.append(word+'@@@'+str(wordres))
            result2.append(wordrank)
            
res_dict = dict(zip(result,result2))
res_dict = sorted(res_dict.items(),key = lambda x:x[1],reverse = True)

In [30]:
result = []
for item in res_dict:
    temp = []
    temp.append(item[0].split("@@@")[0])
    temp.append(item[1])
    temp.append(int(item[0].split("@@@")[1]))
    result.append(temp)
result

[['彩票', 10252, 895],
 ['足彩', 9004, 1003],
 ['足球', 8970, 1187],
 ['体彩', 8891, 669],
 ['直播', 8264, 1394],
 ['网易', 7909, 1502],
 ['福利彩票', 7782, 577],
 ['足球彩票', 7730, 1047],
 ['球探', 7603, 240],
 ['六合彩', 7267, 762],
 ['pk10', 6901, 904],
 ['赛车', 6866, 1211],
 ['体育', 6833, 1434],
 ['双色球', 6807, 775],
 ['博彩', 6770, 830],
 ['竞猜', 6632, 1257],
 ['篮球', 6395, 1428],
 ['福彩', 6304, 594],
 ['天天', 6249, 1464],
 ['新浪', 5860, 1341],
 ['比分', 5659, 1164],
 ['六合', 5342, 806],
 ['3d', 5146, 1434],
 ['福利', 4786, 1020],
 ['预测', 4619, 1056],
 ['大师', 4613, 1415],
 ['啊啊啊', 4605, 109],
 ['开奖', 4605, 794]]

In [31]:
time.time()-t1

10.097028732299805